In [1]:
%%time
import time
time.sleep(2)
# Provides ways to work with large multidimensional arrays
import numpy as np 

# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline


# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

# import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

from IPython.display import display
# Setting display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

import os
import sys
import seaborn as sns
# from pathlib import Path
from dataset.us_equity_load import *
from utils.us_equity_utils import *

from factor.xq_finance import *
from factor.index import *
from dataset.us_equity_load import *
from factor.price import*
# %load_ext ipython_magic
# %load_ext autoreload
# %reload_ext autoreload
# %autoreload 2
# %load_ext ipython_magic
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.instance().extension_manager.load_extension("autoreload")
# InteractiveShell.instance().extension_manager.load_extension("writefile")
import sys
print(sys.executable)


# import IPython
# print(IPython.__version__)
# from IPython.core.magic import register_line_cell_magic

# @register_line_cell_magic
# def writefile(line, cell=None):
#     with open(line, 'w') as f:
#         f.write(cell or '')


folder_name = './fator'
file_name = 'index.py'
# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
file_path = os.path.join('./', 'factor', 'index.py')

/opt/openbb/bin/python
CPU times: user 1.21 s, sys: 1.68 s, total: 2.9 s
Wall time: 2.73 s


In [2]:
# %%writefile $file_path

from dataset.us_equity_load import *

class index:
  def __init__(self, sectors, start_date, end_date, dir = 'fh'):
     self.sectors = sectors
     self.start_date = start_date
     self.end_date = end_date
     self.dir = dir

  def calc(self):
      dict_index = {}
      for sector in self.sectors:
        print(f"processing {sector} ...")
        data_symbols = us_dir1_load_csv(dir0 = 'symbol', dir1 = self.dir, filename= sector +'.csv')
        if (data_symbols.empty == False):
          symbols = data_symbols['symbol'].values

          data = us_equity_daily_data_load(symbols = symbols, start_date = self.start_date,
                                            end_date = self.end_date, trade_option = 'market_capital', 
                                            dir_option = 'xq')
          if (len(data) > 0):
            df = pd.DataFrame(data)
            df_sum = df.sum(axis=1)
            index = df_sum * 1000 /df_sum.iloc[0]
            dict_index[sector] = index
          else:
            self.sectors.remove(sector)
        else:
           print(f"remove {sector} ...")
           self.sectors.remove(sector)

      return pd.DataFrame(dict_index)

  def ratio(self, days = 1):
     df = self.calc()
     df = df.fillna(0)
     return df.pct_change(days).round(5)

# xueqiu sector

In [3]:
sector_file = 'us_equity_sector.csv'
sectors = list(us_dir1_load_csv(dir0 = 'symbol', dir1 = 'xq', filename=sector_file)['name'].values)

start_date = '2020-01-01'
end_date = '2024-08-15'
# sectors = ["Semiconductors_Equipment", "Automobiles", "Internet_Retail", "Electronic_Equipment_Instruments_Components"]
# sectors = ["Semiconductors_&_Semiconductor_Equipment"]
# sectors = ["互联网与直销零售"]
print(sectors)

df_index = index(sectors, start_date, end_date, dir = 'xq')
df = df_index.ratio()

print(df)
us_dir1_store_csv(dir0 = 'symbol', dir1 = 'xq', filename='xq_index_price_ratio' + '.csv', data = df)

['能源设备与服务', '石油、天然气与消费用燃料', '化学制品', '建筑材料', '容器与包装', '金属与采矿', '纸类与林业产品', '航空航天与国防', '建筑产品', '建筑与工程', '电气设备', '工业集团企业', '机械制造', '贸易公司与经销商', '商业服务与商业用品', '专业服务', '航空货运与物流', '航空公司', '海运', '公路与铁路', '交通基本设施', '汽车零配件', '汽车', '家庭耐用消费品', '休闲设备与用品', '纺织品、服装与奢侈品', '酒店、餐馆与休闲', '综合消费者服务', '经销商', '互联网与直销零售', '多元化零售', '专营零售', '食品与主要用品零售', '饮料', '食品', '烟草', '居家用品', '个人用品', '医疗保健设备与用品', '医疗保健提供商与服务', '医疗保健技术', '生物科技', '制药', '生命科学工具和服务', '商业银行', '互助储蓄银行与抵押信贷', '综合金融服务', '消费信贷', '资本市场', '抵押房地产投资信托（REITs）', '保险', '信息技术服务', '软件', '通信设备', '电脑与外围设备', '电子设备、仪器和元件', '半导体产品与设备', '综合电信业务', '无线电信业务', '媒体', '娱乐', '互动媒体与服务', '电力公用事业', '燃气公用事业', '复合型公用事业', '水公用事业', '独立电力生产商与能源贸易商', '股权房地产投资信托 （REITs）', '房地产管理和开发', '工业REIT', '酒店及度假村REIT', '写字楼REIT', '医疗保健REIT', '住宅REIT', '零售REIT', '专门REIT', '房地产管理和开发']
processing 能源设备与服务 ...
lack of some trade date skip  XPRO
lack of some trade date skip  ESOA
lack of some trade date skip  ACDC
lack of some trade date skip  CHX
lack of some trade date skip  RNGR
lack of some trade d

: 

# fh sector

In [4]:
sector_file = 'us_equity_sector.csv'
sectors = list(us_dir1_load_csv(dir0 = 'symbol', dir1 = 'fh', filename=sector_file)['Sector'].values)

start_date = '2022-01-01'
end_date = '2024-08-07'
# sectors = ["Semiconductors_Equipment", "Automobiles", "Internet_Retail", "Electronic_Equipment_Instruments_Components"]
# sectors = ["Semiconductors_&_Semiconductor_Equipment"]
# sectors = ["Diversified_Consumer_Services"]
print(sectors)

df_index = index(sectors, start_date, end_date)
df = df_index.ratio()

print(df)
us_dir1_store_csv(dir0 = 'symbol', dir1 = 'fh', filename='fh_index_price_ratio' + '.csv', data = df)

['Aerospace_&_Defense', 'Air_Freight_&_Logistics', 'Airlines', 'Auto_Components', 'Automobiles', 'Banks', 'Beverages', 'Biotechnology', 'Building_Products', 'Capital_Markets', 'Chemicals', 'Commercial_Services_&_Supplies', 'Communications_Equipment', 'Construction_&_Engineering', 'Construction_Materials', 'Consumer_Finance', 'Distributors', 'Diversified_Consumer_Services', 'Diversified_Financial_Services', 'Diversified_Telecommunication_Services', 'Electric_Utilities', 'Electrical_Equipment', 'Electronic_Equipment,_Instruments_&_Components', 'Energy_Equipment_&_Services', 'Entertainment', 'Equity_Real_Estate_Investment_Trusts_(REITs)', 'Food_&_Staples_Retailing', 'Food_Products', 'Gas_Utilities', 'Health_Care_Equipment_&_Supplies', 'Health_Care_Providers_&_Services', 'Health_Care_Technology', 'Hotels,_Restaurants_&_Leisure', 'Household_Durables', 'Household_Products', 'IT_Services', 'Independent_Power_and_Renewable_Electricity_Producers', 'Industrial_Conglomerates', 'Insurance', 'Inter

In [11]:
print(df)

                     Aerospace_&_Defense  Air_Freight_&_Logistics  Airlines  \
date                                                                          
2024-01-29 05:00:00                  NaN                      NaN       NaN   
2024-01-30 05:00:00                 -1.7                     -4.0      -0.7   
2024-01-31 05:00:00                  0.8                     -1.7      -1.2   
2024-02-01 05:00:00                  0.6                      0.0       1.7   
2024-02-02 05:00:00                 -0.2                      0.5       0.6   
2024-02-05 05:00:00                 -0.4                     -1.1      -1.9   
2024-02-06 05:00:00                  0.8                      3.1       3.0   
2024-02-07 05:00:00                  0.7                      1.2       0.2   
2024-02-08 05:00:00                 -0.7                     -0.1       1.0   
2024-02-09 05:00:00                  0.1                     -0.3       0.6   
2024-02-12 05:00:00                  0.1            